In [26]:
import json
import pandas as pd
from app_store_scraper import AppStore
from google_play_scraper import app, Sort, reviews, reviews_all

## Scrape Apple app store

In [10]:
# Enter app name and app id (found in website url)
data = AppStore(country='us', app_name='vampr-find-artists-gigs', app_id = '1069819177')

# Enter the number of reviews to extract
data.review(how_many=2000)

2024-05-30 11:42:43,645 [INFO] Base - Initialised: AppStore('us', 'vampr-find-artists-gigs', 1069819177)
2024-05-30 11:42:43,646 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/vampr-find-artists-gigs/id1069819177


In [14]:
# View the returned JSON
data.reviews

[{'date': datetime.datetime(2017, 10, 14, 19, 50, 53),
  'developerResponse': {'id': 1613453,
   'body': "Sorry we disappointed you. There's a lot of folks working hard to improve this daily. Good luck with your songwriting. Josh",
   'modified': '2017-10-30T00:08:58Z'},
  'review': 'I set it to global. Swiped on about 5 people, messaged them about their music and it says that they responded but there are no messages. Turns out people just share music that they didn’t even produce/write/mix/master Then the app crashes when I try to save my bio after editing. And then it says it’s loading and trying to discover and it’s been doing that for 2 days. Haven’t seen anybody else pop up on the app, even after I set my preferences every which was. Relaunched app. Hard reset my phone. Nothing. Too many bugs. I’ve got music to write, can’t waste my time on troubleshooting. If this gets fixed I’ll try it again. iPhone 7 Verizon. Sorry guys.',
  'rating': 4,
  'isEdited': False,
  'userName': 'Pen1

In [44]:
# Convert to dataframe, reformat columns, and save as JSON
def create_appstore_metadata(row):
    json_data = {
        "DevResponse": row['developerResponse'],
        "Edited": row['isEdited'],
        "AuthorName": row['userName']
    }
    return json.dumps(json_data)

appstore_df = pd.DataFrame.from_records(data.reviews)

appstore_df["CompanyName"] = "Vampr"
appstore_df["Content"] = appstore_df["title"] + ": " + appstore_df["review"]
appstore_df["Source"] = 'Apple App Store'
appstore_df['ContentType'] = 'Review'
appstore_df = appstore_df.rename(columns={"date": "CreatedDate", "rating": "StarRating"})
appstore_df["CreatedDate"] = pd.to_datetime(appstore_df["CreatedDate"]).astype(str)
appstore_df['Metadata'] = appstore_df.apply(create_appstore_metadata, axis=1)
appstore_df = appstore_df.drop(columns=["title", "review", "developerResponse", "isEdited", "userName"])

appstore_df.to_json('/Users/j/Downloads/Papaya/git_repo/Vetting project/Scraped data/appstore_data/vampr.json')
appstore_df

,CreatedDate,StarRating,CompanyName,Content,Source,ContentType,Metadata
0,2017-10-14 19:50:53,4,Vampr,High hopes. Very very disappointed: I set it t...,Apple App Store,Review,"{""DevResponse"": {""id"": 1613453, ""body"": ""Sorry..."
1,2022-07-22 01:28:59,1,Vampr,Does nothing for normal everyday people.: This...,Apple App Store,Review,"{""DevResponse"": {""id"": 31123884, ""body"": ""Hi t..."
2,2022-06-17 07:25:13,3,Vampr,Bad execution: This app is a great idea and as...,Apple App Store,Review,"{""DevResponse"": NaN, ""Edited"": false, ""AuthorN..."
3,2023-12-16 20:14:44,2,Vampr,Good concept but…: The issues that I can’t get...,Apple App Store,Review,"{""DevResponse"": NaN, ""Edited"": false, ""AuthorN..."
4,2022-11-11 19:29:52,2,Vampr,Not what musicians want: Disappointed to leave...,Apple App Store,Review,"{""DevResponse"": NaN, ""Edited"": false, ""AuthorN..."
...,...,...,...,...,...,...,...
526,2017-08-02 22:39:43,3,Vampr,Has potential but linked tracks need work: Unl...,Apple App Store,Review,"{""DevResponse"": {""id"": 938399, ""body"": ""Hey th..."
527,2017-07-30 12:26:53,3,Vampr,Dope but needs work: Beside the issues below h...,Apple App Store,Review,"{""DevResponse"": {""id"": 938407, ""body"": ""Hey th..."
528,2017-06-01 19:35:40,5,Vampr,Facebook login as artist?: Great idea. However...,Apple App Store,Review,"{""DevResponse"": {""id"": 1067882, ""body"": ""Both ..."
529,2017-06-11 16:02:31,2,Vampr,Idea is alright but app doesn't work properly:...,Apple App Store,Review,"{""DevResponse"": {""id"": 544466, ""body"": ""We can..."


## Scrape Google play reviews

In [24]:
# Get app info

result = app(
    'me.vampr.android', # Enter app id (can be found in app URL)
    lang='en', # defaults to 'en'
    country='us' # defaults to 'us'
)

pretty_json = json.dumps(result, indent=4)
print(pretty_json)

{
    "title": "Vampr - Find & Meet Musicians",
    "description": "Vampr is the #1 social network for artists that helps you find people to collaborate with.\r\n\r\nAward-winning app: Best of App Store and Fast Company: Most Innovative Companies\r\n\r\nOver 1M users and 8M connections.\r\n\r\nVampr has been dubbed the LinkedIn for musicians as it shares each user\u2019s expertise as well as a social media interface with an easy swipe left/right function to connect with others.\r\n- Forbes\r\n\r\nVampr gives user-friendly support for all those songwriters, indie musicians, DJs, rappers, and folk artists, who are struggling to connect and break into the necessary circles they need to.\r\n- HuffPost\r\n\r\nVampr is positioning itself as a LinkedIn for musicians, helping them find one another, collaborate, and even distribute their work.\r\n- FastCompany\r\nWHY VAMPR?\r\n\r\nVampr is the number one social platform helping creatives and artists find people to collaborate with, create new m

In [27]:
result = reviews_all(
    'me.vampr.android',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

In [43]:


# Convert to dataframe, reformat columns, and save as JSON
def create_playstore_metadata(row):
    json_data = {
        "ReviewID": row['reviewId'],
        "AuthorName": row['userName'],
        "AuthorIcon": row['userImage'],
        "ThumbsUpCount": row["thumbsUpCount"],
        "AppVersion": row["appVersion"],
        "DevResponse": row["replyContent"],
        "DevResponseDate": row["repliedAt"]
    }
    return json.dumps(json_data)

playstore_df = pd.DataFrame.from_records(result)

playstore_df["CompanyName"] = "Vampr"
playstore_df["Source"] = 'Google Play'
playstore_df['ContentType'] = 'Review'
playstore_df = playstore_df.rename(columns={"at": "CreatedDate", "score": "StarRating", "content": "Content"})
playstore_df["CreatedDate"] = pd.to_datetime(playstore_df["CreatedDate"]).astype(str)
playstore_df["repliedAt"] = pd.to_datetime(playstore_df["repliedAt"]).astype(str)
playstore_df['Metadata'] = playstore_df.apply(create_playstore_metadata, axis=1)
playstore_df = playstore_df.drop(columns=["reviewId", "userName", "userImage", "thumbsUpCount", "replyContent", "repliedAt", "appVersion", "reviewCreatedVersion"])

playstore_df.to_json('/Users/j/Downloads/Papaya/git_repo/Vetting project/Scraped data/playstore_data/vampr.json')
playstore_df

,Content,StarRating,CreatedDate,CompanyName,Source,ContentType,Metadata
0,Hopefully 1 day!!!,3,2024-05-29 00:26:31,Vampr,Google Play,Review,"{""ReviewID"": ""4800708b-650d-4bce-8979-072d6939..."
1,Perfect way to build your band,5,2024-05-28 13:30:59,Vampr,Google Play,Review,"{""ReviewID"": ""9f7fa752-e013-4754-951e-70032af5..."
2,"No musicians in my area, only ads to pay to go...",1,2024-05-22 16:31:19,Vampr,Google Play,Review,"{""ReviewID"": ""b7ce61b1-6969-4b65-9aa1-b1989b9c..."
3,Conected with over 20 artist and collaborated 🥰,5,2024-05-20 11:31:25,Vampr,Google Play,Review,"{""ReviewID"": ""bb6364cc-53b3-4811-ae85-fa1d8cc7..."
4,Perfect way to connect.,5,2024-05-19 05:49:49,Vampr,Google Play,Review,"{""ReviewID"": ""72c3f525-ed6b-4c87-b550-47e7ed29..."
...,...,...,...,...,...,...,...
791,I like this apps,5,2021-02-24 18:31:23,Vampr,Google Play,Review,"{""ReviewID"": ""63cbe894-891c-4a3b-9cad-bb4a7dff..."
792,I cant open the app,1,2021-02-24 04:15:10,Vampr,Google Play,Review,"{""ReviewID"": ""79869b83-390c-4ab2-bd35-c6486367..."
793,"What the hell is this? No networking on it, no...",1,2021-02-23 02:00:07,Vampr,Google Play,Review,"{""ReviewID"": ""c68032e0-76bc-4f63-b18a-ec0bbe0e..."
794,"Very good, still in development but i think yo...",5,2021-02-22 23:56:50,Vampr,Google Play,Review,"{""ReviewID"": ""ee7918dd-8f6d-4e55-8f9a-46620fe1..."
